# Model for Nature Conservancy Fisheries Kaggle Competition

#### Dependencies

In [1]:
import fish_data as fd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd

#### Helper functions

In [2]:
help(fd)

Help on module fish_data:

NAME
    fish_data

DESCRIPTION
    fish_data module contains the helper functions for the model build of the
    Nature Conservancy Fisheries Kaggle Competition.
    
    Dependencies:
        * numpy as np
        * os
        * scipy.ndimage as ndimage
        * scipy.misc as misc
        * scipy.special as special
        * matplotlib.pyplot as plt
        * tensorflow as tf

FUNCTIONS
    count_nodes(y_in, x_in, conv_depths, conv_strides, pool_strides)
        Calculates the number of total nodes present in the last layer of a
        convolution plus max_pooling architecture.  Calculations assume that
        convolution is 'SAME' padded, and pooling is 'VALID' padded.
    
    decode_image(image_read, size, num_channels=3, mutate=False, brightness_delta=None, contrast_limits=None, hue_delta=None, saturation_limits=None)
        Converts a dequeued image read from filename to a single tensor array,
        with modifications:
            * resized to st

#### Generate a list of filenames

In [3]:
fish_filenames = fd.generate_filenames_list('data/train/', subfolders = True)
print("There are {} filenames in the master set list".format(len(fish_filenames)))
test_filenames = fd.generate_filenames_list('data/test_stg1/', subfolders = False)
print("There are {} filenames in the test set list".format(len(test_filenames)))

There are 3777 filenames in the master set list
There are 1000 filenames in the test set list


#### Generate the labels for the master set list

In [4]:
fish_label_arr = fd.make_labels(fish_filenames, 'train/', '/img')
fish_label_arr.shape
print("One label per row entry: {}".format(all(np.sum(fish_label_arr, 1) == 1) ))

One label per row entry: True


#### Shuffle and split the master set list into training and validation sets

In [5]:
valid_size = 300
files_train, files_val, y_train, y_val = train_test_split(fish_filenames, fish_label_arr, test_size = valid_size)
print("Validation set size: {}".format(y_val.shape[0]))
print("Training set size: {}".format(y_train.shape[0]))

Validation set size: 300
Training set size: 3477


## Graph and Session Runs

#### Graph parameters

In [6]:
%run -i 'PARAMETERS.py'

#### Session parameters

In [7]:
version_ID = 'v1.3.3.5'

In [8]:
%run -i 'GRAPH.py'

In [9]:
%run -i 'SESSION.py'

Initialized!


To view your tensorboard dashboard summary, run the following on the command line:
tensorboard --logdir='/Users/ccthomps/Documents/Python Files/Kaggle Competitions/Nature Conservancy Fisheries/TB_logs/v1.3.3.5'

Batch number: 1
     Training_mean_cross_entropy: 435.47113037109375
     Valid_mean_cross_entropy: 5430.4384765625
Batch number: 5
     Training_mean_cross_entropy: 557.7343139648438
     Valid_mean_cross_entropy: 864.4822998046875
Batch number: 9
     Training_mean_cross_entropy: 23.945446014404297
     Valid_mean_cross_entropy: 34.095237731933594
Batch number: 11
     Training_mean_cross_entropy: 20.988630294799805
     Valid_mean_cross_entropy: 2.0351929664611816
Batch number: 13
     Training_mean_cross_entropy: 4.480085849761963
     Valid_mean_cross_entropy: 2.0222442150115967
Batch number: 17
     Training_mean_cross_entropy: 20.67257308959961
     Valid_mean_cross_entropy: 2.010714292526245
Batch number: 21
     Training_mean_cross_entropy: 8.01749420166